In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [3]:
train = pd.read_csv('../input/santander-customer-satisfaction/train.csv', encoding='latin-1')
test = pd.read_csv('../input/santander-customer-satisfaction/test.csv', encoding='latin-1')
print(train.shape)
print(test.shape)

(76020, 371)
(75818, 370)


In [4]:
# separate features and target variable in the training data
X = train.drop(columns=["TARGET"])  # Features
y = train["TARGET"]                 # Target variable

In [5]:
# split training data into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#create decision tree classifier
model = DecisionTreeClassifier(max_depth=10, random_state=42)

In [7]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')

print("Cross-Validation Scores:", cv_scores)
print("Mean Score:", cv_scores.mean())


Cross-Validation Scores: [0.95843199 0.9581689  0.95764273 0.95784004 0.95843199]
Mean Score: 0.9581031307550646


In [8]:
#train model on training set
model.fit(X_train, y_train)
y_prediction = model.predict(X_val)

In [9]:
model = DecisionTreeClassifier(max_depth=10, class_weight='balanced', random_state=42)


In [10]:
# 去除常值特徵
constant_columns = [col for col in X.columns if X[col].nunique() == 1]
X = X.drop(columns=constant_columns)


In [11]:
#make predictions on validation set


In [12]:
#evaluate the models performance
accuracy = accuracy_score(y_val, y_prediction)
print("Validation Accuracy",accuracy)
print("\nClassification Report\n", classification_report(y_val, y_prediction) )

Validation Accuracy 0.9579715864246251

Classification Report
               precision    recall  f1-score   support

           0       0.96      1.00      0.98     14597
           1       0.10      0.01      0.01       607

    accuracy                           0.96     15204
   macro avg       0.53      0.50      0.50     15204
weighted avg       0.93      0.96      0.94     15204



In [13]:
model.fit(X_train, y_train)

#predict on the test dataset without target column
test_predictions = model.predict(test)

In [14]:
#save prediction to csv
submission = pd.DataFrame({
    "ID":test["ID"], "TARGET":test_predictions
})
submission.to_csv("submission.csv", index=False)